# ACSE-7 (Inversion and Optimisation)  <a class="tocSkip"></a>

# Homework Lecture 6: Unconstrained Optimisation (solutions) <a class="tocSkip"></a>

<font size="1pt">Some $\LaTeX$ definitions hidden in this cell (double-click to reveal)</font>
$
\renewcommand\vec[1]{\mathbf{#1}}
\newcommand\mat[1]{\underline{\mathbf{#1}}}
\newcommand\R{\mathbb{R}}
\newcommand\todo[1]{\textcolor{red}#1}
$

In [2]:
import numpy as np
import scipy.linalg as sl
import matplotlib.pyplot as plt
# font sizes for plots
plt.rcParams['font.size'] = 16
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Arial', 'Dejavu Sans']

In [3]:
%matplotlib inline

# Optimal Box Shape
A chocolate manufacturer wants to design a 6-sided box: six sides and a regular hexagon as the top and bottom:

<img width=200 src="https://images-na.ssl-images-amazon.com/images/I/41LOrfo46sL._AC_SL1000_.jpg">

For a given volume, what is the optimal ratio between the height $h$, and the radius $r$ of the hexagon in order to minimze the amount of packaging, i.e. to minimize the area of the box. **Hint:** you were given the formula of the area of a regular hexagon in the tutorial for lecture 4.

# Constrained Problem

Consider the following constrained problem:

$$
  \text{minimize}f(x, y) = x^4 + y^4 + 2x^2 \\
  \text{subject to }x^2 + y ^2 = 2
$$

* sketch the feasible region
* is this a convex constrained optimisation problem?
* find the stationary points of the constrained problem

# Derivation of the Trust Region method
Using the techniques from today's lecture, we can derive the formula that modifies the Hessian in the Trust Region method (lecture 3).

1. Given a general quadratic function of the usual form

$$
  f(\vec x) = \frac 12 \vec x^T \mat A\vec x - \vec b^T\vec x +c
$$

find an equation for the minimum $f(\vec x)$ for points $\vec x$ *on* the circle (in 2D) or sphere (in 3D) or "hyper-sphere" in n dimensions using the following equality constraint:

$$
  g(\vec x) = \frac 12\vec x^T \vec x - \frac 12 r^2 = 0
$$

You don't need to solve for the Lagrange multiplier (for this see the "final note" below), but try arrive an equation in the following form:

$$
  \left[\mat A + \lambda\mat I\right]\vec x = 
\vec b
$$

where $\mat I$ is the identity matrix. You may assume that $\mat A$ is SPD to ensure that a minimum exists, and is unique.

2. Now proof the following expression for the optimisation problem of finding the minimum *inside* (or on the boundary of) a circle/sphere/hyper-sphere of radius $r$:

$$
  \left[\mat A + \mu\mat I\right]\vec x = 
\vec b
$$

What conditions do we have for $\mu$?

3. Write down the first three terms in the Taylor series approximation of a function $f:\R^n \to\R$ in a point $\vec x^{(i)}+\vec p$ near another point $\vec x^{(i)}$:

$$
  f(\vec x^{(i)}+\vec p) = ...
$$

Find the minimum of this approximation formed by the three terms (derive an expression to solve for $\vec p$), and relate your result to the (unconstrained) Newton method.

4. In the trust region method, we first try a standard Newton step, which you've derived again in the previous question. If the resulting $\vec p$ does not lead to a sufficient decrease in the new function value $f(\vec x^{(i)}+\vec p)$, we restrict the region in which we *trust* the quadratic approximation. We do this by choosing a radius $r$ that is smaller than the step size of our first try, and find the minimum of the quadratic approximation with the restriction that $\|\vec p\|\leq r$. In other words we choose a sphere of radius $r$ around $\vec x^{(i)}$ in which we trust the approximation. Combine your answers to question 2. and 3. to derive an equation for this trust region approach. Again you may assume that the Hessian in $\vec x^{(i)}$ is SPD. Why can we assume that the minimum is *not* going to be found in the interior of the radius-$r$ sphere?

### Final note <a class="tocSkip"></a>
In our previous answer, like in the answer to 2. we still have a $\mu$ that we don't know. We can derive an equation for $\mu$ by rewriting the result of answer 2 in the form:

$$
  \vec x = \left[\mat A + \mu\mat I\right]^{-1}\vec b
$$

If we assume the minimum is found on the boundary of the sphere, we have

$$
  \vec x^T\vec x = r^2
$$

which we can work out to

\begin{align*}
  \vec b^T \left[\mat A + \mu\mat I\right]^{-1,T}\left[\mat A + \mu\mat I\right]^{-1}\vec b &= r^2 \\
  \vec b^T \left(\left[\mat A + \mu\mat I\right]\left[\mat A + \mu\mat I\right]^T\right)^{-1}\vec b &= r^2
\end{align*}

This is an equation that we can solve for $\mu$. As you can see, it is not a very nice equation to solve. There are therefore various approaches to deal with this situation:

* in an *exact trust region approach* we do solve the equation above. We choose an initial radius $r$, which we may for instance halve every time we find a point that does not satisfy a sufficient decrease condition (e.g. Armijo). We then solve the above equation for $\mu$. It is a one-dimensional nonlinear equation, so in principle we can just solve with for instance Newton. Obviously this might add considerable expense to the algorithm
* in other approaches, instead of choosing a $r$ beforehand, we simply treat $\mu$ as a free parameter. We start using $\mu=0$ which gives plain Newton, then if the sufficient decrease hasn't been achieved we simply increase $\mu$. For very big $\mu$ we get

$$
  \mat A+\mu\mat I\approx \mu\mat I
$$

and therefore we have a update step of

$$
  \mu\mat I\vec p = -f'(\vec x^{(I)}) \implies \vec p =-\frac 1\mu f'(\vec x^{(I)})
$$

which is just Steepest Descent with a "line search parameter" $\lambda=1/\mu$. Thus we can choose any $\nu$ between zero and infinity, where for big $\mu$ we transform the method into Steepest Descent. So we can use this method in a similar way as backtracking line search, but instead of using a $\lambda$ that starts at 1 and is decreased until we satisfy a sufficient decrease condition in $f$, we now have a parameter $\nu$ that we start at zero and keep increasing until the condition is satisfied. For more information see e.g. [Kelley's "Iterative Methods for Optimization"](https://www.amazon.com/Iterative-Methods-Optimization-Frontiers-Mathematics/dp/0898714338).